<a href="https://colab.research.google.com/github/amitganon/Data-engineering/blob/main/314770629_%2B_207298308_%D7%94%D7%A0%D7%93%D7%A1%D7%AA_%D7%A0%D7%AA%D7%95%D7%A0%D7%99%D7%9D_%D7%A2%D7%91%D7%95%D7%93%D7%94_5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os

# Installing the Kaggle package
!pip install kaggle 

!mkdir /root/.kaggle/ 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token ={"username":"amitganon","key":"3d37cfbaf99868ba7700ca65f0bfa80d"}


# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# Creating a dataset directory
!mkdir ./datasets

In [ ]:
# download the dataset from Kaggle and unzip it 
!kaggle datasets download -d rtatman/blog-authorship-corpus

 95% 274M/290M [00:01<00:00, 196MB/s]
100% 290M/290M [00:01<00:00, 192MB/s]


In [ ]:
!unzip /content/blog-authorship-corpus.zip
!ls ./datasets/blog-authorship-corpus

Archive:  /content/blog-authorship-corpus.zip
  inflating: blogtext.csv            y

ls: cannot access './datasets/blog-authorship-corpus': No such file or directory


In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
bt= pd.read_csv('/content/blogtext.csv')
bt

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
681279,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, I could write some really ..."
681280,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, 'I have the second yeast i..."
681281,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, Your 'boyfriend' is fuckin..."
681282,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan: Just to clarify, I am as..."


## Blog Authorship Corpus

## Q1

In [ ]:
bt1=pd.DataFrame()
bt1=bt[['sign','text']].copy()
bt1=bt1.dropna()

def countS2 (str):
    t = re.findall(r'\b\w{8,}\b', str)
    return len(t)
l=[]
for i in tqdm(range(len(bt1))):
  x=countS2(bt1.text[i])
  l.append(x)
bt1['counts']=l

g=bt1.groupby("sign" , as_index=False)['counts'].sum()
g=g.max()
g.sign


'Virgo'

## Q2

In [ ]:
from datetime import datetime
bt['deadline'] = pd.to_datetime(bt['date'],errors='coerce')
bt['dayinweek'] = bt['deadline'].dt.dayofweek
g=bt.groupby('dayinweek').count()
g.text

dayinweek
0.0    115289
1.0    110235
2.0    100156
3.0    104817
4.0     85935
5.0     64305
6.0     92801
Name: text, dtype: int64

## Q3

In [ ]:
bt2=pd.DataFrame()
bt2=bt[['topic','text']]
bt2=bt2.iloc[:100000]

def countMail (str):
    t = re.findall(r'[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,3}', str)
    return len(t)

l2=[]   
for i in tqdm(range(len(bt2))):
  x=countMail(bt2.text[i])
  l2.append(x)

bt2['count']=l2

g=bt2.groupby("topic" , as_index=False)['count'].sum()
g


,topic,count
0,Accounting,0
1,Advertising,12
2,Agriculture,0
3,Architecture,0
4,Arts,25
5,Automotive,0
6,Banking,0
7,Biotech,2
8,BusinessServices,31
9,Chemicals,0


## Q4

In [ ]:
bt3=pd.DataFrame()
bt3=bt[['id','text']]
bt3=bt3.iloc[:100000]

def countS3 (str):
    t = re.findall(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%.\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%\+.~#?&//=]*)', str)
    return len(t)
    
l3=[]
for i in tqdm(range(len(bt3))):
  x=countS3(bt3.text[i])
  l3.append(x)

bt3['counts']=l3

g=bt3.groupby("id" , as_index=False)['counts'].sum()
g=g.max()
g.id


4337133

## Q5

In [ ]:
bt5=pd.DataFrame()
bt5=bt[['topic','text']]
bt5=bt5.iloc[:100000]

def countS5 (str):
    t = re.findall(r'(\d+,{0,1}\d+)', str)
    x=0
    for j in range(len(t)):
      t[j]=t[j].replace(",","")
      if(len(t[j])==0):
         t[j]='0'
      for k in range(len(t)):
        if(len(t[k])>x):
          x=len(t[k])
    if(len(t)==0):
      t='0'
    return x

tempmax=0
tempindex=0
for i in tqdm(range(len(bt5))):
  x=countS5(bt5.text[i])
  if(tempmax<x):
    tempmax=x
    tempindex=i

print(bt5.topic[tempindex],tempmax)



Student 41


## UFO Sightings

In [ ]:
!kaggle datasets download -d NUFORC/ufo-sightings

  0% 0.00/10.2M [00:00<?, ?B/s]
100% 10.2M/10.2M [00:00<00:00, 94.3MB/s]


In [ ]:
!unzip /content/ufo-sightings.zip
!ls ./datasets/

Archive:  /content/ufo-sightings.zip
  inflating: complete.csv            
  inflating: scrubbed.csv            


In [ ]:
scr = pd.read_csv('/content/scrubbed.csv', low_memory=False)
scr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80332 entries, 0 to 80331
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   datetime              80332 non-null  object 
 1   city                  80332 non-null  object 
 2   state                 74535 non-null  object 
 3   country               70662 non-null  object 
 4   shape                 78400 non-null  object 
 5   duration (seconds)    80332 non-null  object 
 6   duration (hours/min)  80332 non-null  object 
 7   comments              80317 non-null  object 
 8   date posted           80332 non-null  object 
 9   latitude              80332 non-null  object 
 10  longitude             80332 non-null  float64
dtypes: float64(1), object(10)
memory usage: 6.7+ MB


In [ ]:
scr = pd.read_csv('/content/scrubbed.csv', low_memory=False)

scr['datetime'] = pd.to_datetime(scr['datetime'],errors='coerce')

## Q1

In [ ]:
scr1 = scr[['datetime','country']].copy()
scr1=scr1.dropna()
scr1['datetime'] = scr1['datetime'].dt.year
scr1.datetime=scr1.datetime.astype('int64')

def ufoCountry(country,year):
  return scr1.set_index(['datetime', 'country']).loc[year,country].shape[0]

x=ufoCountry("us",1994)
x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  import sys


321

## Q2

In [ ]:
k= scr1.groupby(['datetime'],as_index=False).count()
x=k.max()['country']
y=k[k['country']==x]['datetime']
for i in y.values:
  print (i)

2012


## Q3

In [ ]:
scr2 = scr.copy()
scr2['datetime'] = scr2['datetime'].dt.hour
scr2=scr2.dropna()
scr2.datetime = scr2.datetime.astype('int64')

In [ ]:
h = scr2.groupby(['datetime'],as_index=False).count()
max= h.max()['country']
y=h[h['country']==max]['datetime']
for i in y.values:
  print (i)

21


## Q4

In [ ]:
scr3=pd.DataFrame()
scr3=scr[['comments']].copy()
scr3=scr3.fillna("")

def countS3 (str):
    t = re.findall(r'\b\w{6,}\s+\w{6,}\b', str)
    return len(t)

l=[]
for i in tqdm(range(len(scr3))):
  #print(scr3.comments[i])
  x=countS3(scr3.comments[i])
  l.append(x)
scr3['counts']=l

x=scr3[scr3['counts']==scr3.counts.max()]['comments']
print(x)


640      3 oval shaped reddish-orange lights flew over ...
16846    Rising fireball&#44 turned into rounded square...
67625    small circular flashing object emiting differe...
Name: comments, dtype: object


## Q5

In [ ]:
scr4=pd.DataFrame()
scr4=scr[['comments','country']].copy()
scr4=scr4.fillna("")
def countS4 (str):
    t = re.findall(r'\D(\d{4})\D', str)
    return len(t)

l=[]
for i in tqdm(range(len(scr4))):
  #print(scr3.comments[i])
  x=countS4(scr4.comments[i])
  l.append(x)
scr4['counts']=l

x=scr4[scr4['counts']==scr4.counts.max()]['country']
print(x)


60019    us
Name: country, dtype: object


## Q6

In [ ]:
scr5=pd.DataFrame()
scr=scr.fillna("")
for i in tqdm(range(len(scr))):
  t = re.findall(r'\b\w\w\d\w\b', scr.comments[i])
  if len(t) > 0:
    print(scr.loc[i])


Streaming output truncated to the last 5000 lines.
duration (seconds)                                                    180
duration (hours/min)                                            3 minutes
comments                what caught my attention of the evening of 13 ...
date posted                                                    10/28/2002
latitude                                                       48.1183333
longitude                                                        -123.429
Name: 1282, dtype: object
datetime                                              2003-10-13 21:30:00
city                                                               warsaw
state                                                                  in
country                                                                us
shape                                                               other
duration (seconds)                                                    180
duration (hours/min)               